In [1]:
# Imports
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline     
import pandas as pd
import numpy as np
import glob,os
from tqdm.auto import tqdm
import math
import sklearn
from sklearn.linear_model import LinearRegression

import tigramite
from tigramite import data_processing as pp
from tigramite.toymodels import structural_causal_processes as toys
from tigramite.models import Models, Prediction
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb

/home/saranya/anaconda3/envs/bmap/lib/python3.8/site-packages/tigramite/models.py:29: UserWarning: [Errno 2] No such file or directory: '/home/saranya/anaconda3/envs/bmap/lib/python3.8/site-packages/tigramite/../versions.py'
  warnings.warn(str(e))
/home/saranya/anaconda3/envs/bmap/lib/python3.8/site-packages/tigramite/plotting.py:26: UserWarning: [Errno 2] No such file or directory: '/home/saranya/anaconda3/envs/bmap/lib/python3.8/site-packages/tigramite/../versions.py'
  warnings.warn(str(e))
/home/saranya/anaconda3/envs/bmap/lib/python3.8/site-packages/tigramite/independence_tests/gpdc.py:27: UserWarning: [Errno 2] No such file or directory: '/home/saranya/anaconda3/envs/bmap/lib/python3.8/site-packages/tigramite/independence_tests/../../versions.py'
  warnings.warn(str(e))
/home/saranya/anaconda3/envs/bmap/lib/python3.8/site-packages/tigramite/independence_tests/gpdc_torch.py:33: UserWarning: No module named 'torch'
  warnings.warn(str(e))


In [2]:
def select_links(tau_min, tau_max, parents, children):
    """
    This function selects the causal links that will be tested by
    PCMCI. The links are selected such that per each variable in
    `children` all `parents` are stablished as causes, and no other
    causal relationships exist.
    
    Assumes `parents` and `children` are disjoint sets, and that all
    variables are included in the union of both sets.
    
    Parameters
    ----------
    tau_min : int
        Minimum time lag to test. Note that zero-lags are undirected.
    tau_max : int
        Maximum time lag. Must be larger or equal to tau_min.
    parents : set of int
        List of variables that will be assigned as a parent link.
        Assumed to be disjoint with children
    children : set of int
        List of variables that will be assigned a link from a parent.
        Assumed to be disjoint with parents
    Returns
    -------
    selected_links: dict
        Dictionary of selected links for Tigramite
        
    """

    parents = set(parents)
    children = set(children)

    selected_links = dict()
    # Set the default as all combinations of the selected variables
    for var in [*children, *parents]:
        if var in children:
            # Children can be caused only by parents and by themselves
            selected_links[var] = [
                (parent, -lag)
                for parent in parents
                for lag in range(tau_min, tau_max + 1)
            ]
        else:
            selected_links[var] = []

    return selected_links

def _process_dataset(path=None):
    df1 = pd.read_csv(path,sep=',')
    df1.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
    df1=df1.drop('a', axis=1)
    df1=df1.drop('conv_rrate', axis=1)
    df1=df1.drop('ls_rrate', axis=1)
    df1=df1.drop('mn_conv_prate', axis=1)
    df1=df1.drop('mn_ls_prate', axis=1)
    df1=df1.drop('mn_tot_prate', axis=1)
    df1=df1.drop('outconv_rrate', axis=1)
    df1=df1.drop('outls_rrate', axis=1)
    df1=df1.drop('outmn_conv_prate', axis=1)
    df1=df1.drop('outmn_ls_prate', axis=1)
    df1=df1.drop('outmn_tot_prate', axis=1)
    df1=df1.drop('conv_ppt', axis=1)
    df1=df1.drop('outconv_ppt', axis=1)
    
    TCname = path.split('/')[-1].split('.')[0].split('_')[-1]
    #print(TCname)
    for item in glob.glob('/media/saranya/DAEA17D6EA17ADAF/Data/Research_2022/jun22/targets/*tot_ppt_int*'):
        if str(TCname) in item:
            d1=pd.read_csv(item)
            d1.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
            d1=d1.drop('a', axis=1)
            dt1=pd.concat([d1,df1],axis=1, join='inner')
        else:
            continue
    return dt1

In [3]:
p1="../../tigramite_timeseries/"
p2="../../targets/"

dt1=_process_dataset(glob.glob(p1+'*keila*')[0])
dt2=_process_dataset(glob.glob(p1+'*viyaru*')[0])
dt3=_process_dataset(glob.glob(p1+'*phailin*')[0])
dt4=_process_dataset(glob.glob(p1+'*lehar*')[0])
dt5=_process_dataset(glob.glob(p1+'*madi*')[0])
dt6=_process_dataset(glob.glob(p1+'*nanauk*')[0])
dt7=_process_dataset(glob.glob(p1+'*hudhud*')[0])
dt8=_process_dataset(glob.glob(p1+'*nilofar*')[0])
dt9=_process_dataset(glob.glob(p1+'*ashoba*')[0])
dt10=_process_dataset(glob.glob(p1+'*chapala*')[0])
dt11=_process_dataset(glob.glob(p1+'*megh*')[0])
dt12=_process_dataset(glob.glob(p1+'*roanu*')[0])
dt13=_process_dataset(glob.glob(p1+'*kyant*')[0])
dt14=_process_dataset(glob.glob(p1+'*vardah*')[0])
dt15=_process_dataset(glob.glob(p1+'*ockhi*')[0])
dt16=_process_dataset(glob.glob(p1+'*luban*')[0])
dt17=_process_dataset(glob.glob(p1+'*gaja*')[0])
dt18=_process_dataset(glob.glob(p1+'*fani*')[0])
dt19=_process_dataset(glob.glob(p1+'*vayu*')[0])
dt20=_process_dataset(glob.glob(p1+'*kyaar*')[0])
dt21=_process_dataset(glob.glob(p1+'*maha*')[0])
dt22=_process_dataset(glob.glob(p1+'*bulbul*')[0])
dt23=_process_dataset(glob.glob(p1+'*pawan*')[0])
dt24=_process_dataset(glob.glob(p1+'*amphan*')[0])

In [4]:
tcnio1=dt1.values
tcnio2=dt2.values
tcnio3=dt3.values
tcnio4=dt4.values
tcnio5=dt5.values
tcnio6=dt6.values
tcnio7=dt7.values
tcnio8=dt8.values
tcnio9=dt9.values
tcnio10=dt10.values
tcnio11=dt11.values
tcnio12=dt12.values
tcnio13=dt13.values
tcnio14=dt14.values
tcnio15=dt15.values
tcnio16=dt16.values
tcnio17=dt17.values
tcnio18=dt18.values
tcnio19=dt19.values
tcnio20=dt20.values
tcnio21=dt21.values
tcnio22=dt22.values
tcnio23=dt23.values
tcnio24=dt24.values

ddnio={'cyclone_nio1':tcnio1,'cyclone_nio2':tcnio2,'cyclone_nio3':tcnio3,'cyclone_nio4':tcnio4,'cyclone_nio5':tcnio5,'cyclone_nio6':tcnio6,
      'cyclone_nio7':tcnio7, 'cyclone_nio8':tcnio8,'cyclone_nio9':tcnio9,'cyclone_nio10':tcnio10,'cyclone_nio11':tcnio11,'cyclone_nio12':tcnio12,
      'cyclone_nio13':tcnio13,'cyclone_nio14':tcnio14,'cyclone_nio15':tcnio15,'cyclone_nio16':tcnio16,'cyclone_nio17':tcnio17,'cyclone_nio18':tcnio18,
      'cyclone_nio19':tcnio19,'cyclone_nio20':tcnio20,'cyclone_nio21':tcnio21,'cyclone_nio22':tcnio22,'cyclone_nio23':tcnio23,'cyclone_nio24':tcnio24}

In [18]:
list(ddnio.keys())[0][11:]

'1'

In [22]:
class Pipeline:
    """
    Tigramite and Linear Regression Pipeline
    """
    def __init__(self,data,pc_alpha,alpha_level,pc_type='run_pcstable' or 'pcmci',tau_min0=None,tau_max0=None,
                 target='precip',var_name=None,seed=None,cond_ind_test=ParCorr()):
        self.pc_alpha = pc_alpha
        self.alpha_level = alpha_level
        self.data = data
        self.pc_type = pc_type
        self.target = target
        self.tau_min0 = tau_min0
        self.tau_max0 = tau_max0
        self.var_name = var_name
        self.seed = seed
        self.cond_ind_test = cond_ind_test
        
    #################################################################################
    # Step 0: Split
    #################################################################################
    def splitdata(self,testindex=None,basin='WPAC'):
        datae = self.data.copy()
        traindata = {}
        testdata = {}
        validdata = {}
        validindex,newtestindex = testindex[:int(len(testindex)/2)],testindex[int(len(testindex)/2):]
        for obj in datae.keys():
            if basin=='WPAC':
                number = int(obj[7:])
                if number in list(newtestindex):
                    testdata['cyclone'+str(number)] = datae['cyclone'+str(number)]
                elif number in list(validindex):
                    validdata['cyclone'+str(number)] = datae['cyclone'+str(number)]
                else:
                    traindata['cyclone'+str(number)] = datae['cyclone'+str(number)]
            elif basin=='NIO':
                number = int(obj[11:])
                if number in list(newtestindex):
                    testdata['cyclone_nio'+str(number)] = datae['cyclone_nio'+str(number)]
                elif number in list(validindex):
                    validdata['cyclone_nio'+str(number)] = datae['cyclone_nio'+str(number)]
                else:
                    traindata['cyclone_nio'+str(number)] = datae['cyclone_nio'+str(number)]
        return traindata,validdata,testdata
    
    #################################################################################
    # Step 1: Tigramite
    #################################################################################
    def run_tigramite(self,lengthtrain=None):
        #assert len(self.data)==lengthtrain,"Wrong shape!"
        datae = self.data.copy()
        dataframe =  pp.DataFrame(datae,analysis_mode ='multiple', var_names=var_names)
        #################################################################################
        # Sel links
        #################################################################################
        for member in dataframe.values.keys():
            children = [0,1,2]
            parents = np.arange(3,259)
            sel_links = select_links(self.tau_min0, self.tau_max0, parents, children)
        #################################################################################
        # Run Tigramite
        #################################################################################        
        pcmci = PCMCI(dataframe = dataframe, cond_ind_test = self.cond_ind_test)
        if self.pc_type=='run_pcstable':
            results = pcmci.run_pcstable(selected_links=sel_links, tau_min=self.tau_min0, tau_max=self.tau_max0,\
                                          pc_alpha=self.pc_alpha)
        elif self.pc_type=='pcmci':
            results = pcmci.run_pcmci(selected_links=sel_links, tau_min=self.tau_min0, tau_max=self.tau_max0,\
                                      pc_alpha=self.pc_alpha)

        pcmci.verbosity = 2
        #################################################################################
        # Test
        #################################################################################   
        #pcmci.print_results(results,self.alpha_level)
        #pcmci.print_significant_links(p_matrix=results['p_matrix'],
        #val_matrix = results['val_matrix'],
        #alpha_level = self.alpha_level)
        return results
    
    #################################################################################
    # Step 2: Linear Regression
    #################################################################################
    # Helper functions
    #################################################################################
    def random_testindex(self,totalexp=None,testexp=None):
        from numpy.random import default_rng
        rng = default_rng(self.seed)
        seed = rng.choice(totalexp, testexp, replace=False)
        return seed
    
    def extract_lag_info(self,datar=None,varindex=None,lag=None):
        temp = datar[:,varindex] # Full time series
        store = []
        for timeindex in range(len(temp)):
            if timeindex < np.abs(lag):
                store.append(np.nan)
            elif timeindex > len(temp)-1-np.abs(lag):
                store.append(np.nan)
            else:
                store.append(temp[timeindex-np.abs(lag)])
        return store
    
    def extract_var_and_lag(self,pcmci_results=None,p_or_q='p' or 'q'):
        datae = self.data.copy()
        dataframe =  pp.DataFrame(datae,analysis_mode ='multiple', var_names=var_names)
        pcmci = PCMCI(dataframe = dataframe, cond_ind_test = self.cond_ind_test)
        #################################################################################
        # Sel links
        #################################################################################
        for member in dataframe.values.keys():
            children = [0,1,2]
            parents = np.arange(3,259)
            sel_links = select_links(self.tau_min0, self.tau_max0, parents, children)
        
        q_matrix = pcmci.get_corrected_pvalues(p_matrix=pcmci_results['p_matrix'],
                                               selected_links = sel_links, 
                                               tau_min=self.tau_min0,tau_max=self.tau_max0, fdr_method='fdr_bh')
        #################################################################################
        # Save vars and lags
        #################################################################################
        if p_or_q=='p':
            sig_links = (pcmci_results['p_matrix'].copy() <= self.alpha_level)
        elif p_or_q=='q':
            sig_links = (q_matrix.copy() <= self.alpha_level)
        arr = []
        for j in range(3):
            links = {(p[0], -p[1]): np.abs(pcmci_results['val_matrix'][p[0],j,abs(p[1])]) for p in zip(*np.where(sig_links[:,j,:]))}
            # Sort by value
            sorted_links = sorted(links, key=links.get, reverse=True)
            arr.append(sorted_links)
        return arr
    
    #################################################################################
    # Process functions
    #################################################################################
    def preproc_ts_withnan(self,links=None,group=None,trainmean=None,trainstd=None):
        if self.target=='precip':
            arrtarget,ytarget = links[0],[self.data[obj][:,0] for obj in self.data.keys()]
        elif self.target=='pmin':
            arrtarget,ytarget = links[1],[self.data[obj][:,1] for obj in self.data.keys()]
        elif self.target=='v10':
            arrtarget,ytarget = links[2],[self.data[obj][:,2] for obj in self.data.keys()]
        
        # 1. Extract time series with nan
        varindexstore,lagstore,TS_store,flatTS_store = [],[],[],[]
        for varindex,lag in arrtarget:
            varindexstore.append(varindex)
            lagstore.append(lag)
            tempp = [self.extract_lag_info(datar=self.data[obj],varindex=varindex,lag=lag) for obj in self.data.keys()]
            TS_store.append(tempp)
            flatTS_store.append(np.concatenate([obj for obj in tempp]))
        
        # 2. Normalize
        if group=='train':
            TSnorml_store,meanstore,stdstore = [],[],[]
            for i in range(len(TS_store)):
                tempmean,tempstd = np.nanmean(flatTS_store[i]),np.nanstd(flatTS_store[i])
                TSnorml_store.append([(obj-tempmean)/tempstd for obj in TS_store[i]])
                meanstore.append(tempmean)
                stdstore.append(tempstd)
        elif (group=='test') or (group=='valid'):
            TSnorml_store = []
            for i in range(len(TS_store)):
                tempmean,tempstd = (trainmean[i]),(trainstd[i])
                TSnorml_store.append([(obj-tempmean)/tempstd for obj in TS_store[i]])
            
        # 3. Concatenate
        #validindex,testindex = validtestindex[0:int(np.round(len(validtestindex)/2))],\
        #validtestindex[int(np.round(len(validtestindex)/2)):]
        
        Xtrain_withnan_store = []
        for i in range(len(TS_store)):
            storelist = [i for j, i in enumerate(TSnorml_store[i])]
            Xtrain_withnan_store.append(np.concatenate([obj for obj in storelist]))
            
        Ytrain = np.concatenate([i for j, i in enumerate(ytarget)])
        if group=='train':
            return Xtrain_withnan_store,Ytrain,meanstore,stdstore
        elif group=='valid':
            return Xtrain_withnan_store,Ytrain
        elif group=='test':
            return Xtrain_withnan_store,Ytrain
    
    def remove_nan(self,Xdict=None,ydict=None):
        def _remove_nan(X=None,y=None):
            X_nonan,Y_nonan = [],[]
            X_withnan_storer = np.asarray(X).transpose()
            for i in (range(len(y))):
                tempX = X_withnan_storer[i,:]
                if np.isnan(tempX).any():
                    continue
                else:
                    X_nonan.append(tempX)
                    Y_nonan.append(y[i])
            return X_nonan,Y_nonan
        Xtrain,ytrain = _remove_nan(X=Xdict['train'],y=ydict['train'])
        Xvalid,yvalid = _remove_nan(X=Xdict['valid'],y=ydict['valid'])
        Xtest,ytest = _remove_nan(X=Xdict['test'],y=ydict['test'])
        return {'train':Xtrain,'valid':Xvalid,'test':Xtest},{'train':ytrain,'valid':yvalid,'test':ytest}

    #################################################################################
    # Train model
    #################################################################################
    def train_mlr(self,X=None,y=None):
        regr = LinearRegression()
        regr.fit(X['train'],y['train'])
        return regr

In [20]:
def trainMLR_target(target='precip' or 'pmin' or 'v10',seed=12345):
    ################################################################################################################
    # 1. X,y with nan
    Xtrain_nan,ytrain_nan,trainmean,trainstd = Pipeline(traindata,pc_alpha,alpha_level,pc_type='pcmci',\
                                                        tau_min0=1,tau_max0=16,\
                                                        target=target,var_name=var_names,\
                                                        seed=seed).preproc_ts_withnan(links=var_and_lag,group='train',\
                                                                                       trainmean=None,trainstd=None)
    Xvalid_nan,yvalid_nan = Pipeline(validdata,pc_alpha,alpha_level,pc_type='pcmci',\
                                   tau_min0=1,tau_max0=16,target=target,\
                                   var_name=var_names,seed=seed).preproc_ts_withnan(links=var_and_lag,\
                                                                                     group='valid',\
                                                                                     trainmean=trainmean,\
                                                                                     trainstd=trainstd)
    
    Xtest_nan,ytest_nan = Pipeline(testdata,pc_alpha,alpha_level,pc_type='pcmci',\
                                   tau_min0=1,tau_max0=16,target=target,\
                                   var_name=var_names,seed=seed).preproc_ts_withnan(links=var_and_lag,\
                                                                                     group='test',\
                                                                                     trainmean=trainmean,\
                                                                                     trainstd=trainstd)
    X_nan = {'train':Xtrain_nan,'valid':Xvalid_nan,'test':Xtest_nan}
    y_nan = {'train':ytrain_nan,'valid':yvalid_nan,'test':ytest_nan}
    ################################################################################################################
    # 2. remove nan
    X_nonan,y_nonan = Pipeline(traindata,pc_alpha,alpha_level,pc_type='pcmci',\
                               tau_min0=1,tau_max0=16,target=target,\
                               var_name=var_names,seed=seed).remove_nan(Xdict=X_nan,ydict=y_nan)
    ################################################################################################################
    # 3. Train MLR
    wpac_mlr = Pipeline(traindata,pc_alpha,alpha_level,pc_type='pcmci',\
                        tau_min0=1,tau_max0=16,target=target,\
                        var_name=var_names,seed=seed).train_mlr(X=X_nonan,y=y_nonan)
    ################################################################################################################
    return wpac_mlr,X_nonan,y_nonan

import pickle
def save_to_pickle(loc=None,var=None):
    with open(loc,"wb") as f:
        pickle.dump(var,f)
    return None

#### Process

In [8]:
len(ddnio)

24

In [23]:
pc_alphaa,alpha_levellist,splitsize,seednum=0.01,[0.2,0.15,0.1,0.01,1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9],6,12348
#[2e-1,1e-1,1e-2,1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9]
var_names=dt1.columns.values.tolist()
for pc_alpha in tqdm([0.2,0.1,0.05,0.01,1e-3,1e-4,1e-5,1e-6,1e-7,1e-8]):
    for alpha_level in (alpha_levellist):
        testindex = (Pipeline(ddnio,pc_alpha,alpha_level,pc_type='pcmci',tau_min0=1,tau_max0=16,\
                              target=None,var_name=var_names,seed=seednum).random_testindex(24,splitsize))
        traindata,validdata,testdata = Pipeline(ddnio,pc_alpha,alpha_level,pc_type='pcmci',tau_min0=1,tau_max0=16,\
                                                target='precip',var_name=var_names,seed=seednum).splitdata(testindex,'NIO')
        result = Pipeline(traindata,pc_alpha,alpha_level,pc_type='pcmci',tau_min0=1,tau_max0=16,\
                          target='precip',var_name=var_names,seed=seednum).run_tigramite(int(24-splitsize))
        var_and_lag = Pipeline(traindata,pc_alpha,alpha_level,pc_type='pcmci',tau_min0=1,tau_max0=16,\
                               target='precip',var_name=var_names,seed=seednum).extract_var_and_lag(result,p_or_q='p')
        wpac_mlr_precip,X_precip,y_precip = trainMLR_target(target='precip',seed=seednum)
        wpac_mlr_pmin,X_pmin,y_pmin = trainMLR_target(target='pmin',seed=seednum)
        wpac_mlr_v10,X_v10,y_v10 = trainMLR_target(target='v10',seed=seednum)
        save_to_pickle(loc='./pickleddata4/'+str(seednum)+'/causalnio_precip.obj.'+\
                       str(pc_alpha)+'.'+str(alpha_level)+'.'+str(seednum),\
                       var={'mlr':wpac_mlr_precip,'X':X_precip,'y':y_precip})
        save_to_pickle(loc='./pickleddata4/'+str(seednum)+'/causalnio_pmin.obj.'+\
                       str(pc_alpha)+'.'+str(alpha_level)+'.'+str(seednum),\
                       var={'mlr':wpac_mlr_pmin,'X':X_pmin,'y':y_pmin})
        save_to_pickle(loc='./pickleddata4/'+str(seednum)+'/causalnio_v10.obj.'+\
                       str(pc_alpha)+'.'+str(alpha_level)+'.'+str(seednum),\
                       var={'mlr':wpac_mlr_v10,'X':X_v10,'y':y_v10})
        save_to_pickle(loc='./pickleddata4/'+str(seednum)+'/causalnio_lag_and_links.'+\
                       str(pc_alpha)+'.'+str(alpha_level)+'.'+str(seednum),var=var_and_lag)

  0%|          | 0/10 [00:00<?, ?it/s]